In [ ]:
import pandas as pd
import os
import glob


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

6개의 데이터 파일 읽어 들이기

In [ ]:


def merge_csv_files(folder_path, encoding='utf-8'):
    """
    지정된 폴더 내의 모든 CSV 파일을 하나의 DataFrame으로 병합합니다.

    Parameters:
        folder_path (str): CSV 파일들이 위치한 폴더 경로
        encoding (str): 파일 인코딩 (기본값: 'utf-8')

    Returns:
        pd.DataFrame: 병합된 데이터프레임
    """
    # CSV 파일 목록 가져오기
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    
    if not csv_files:
        print("CSV 파일이 없습니다.")
        return pd.DataFrame()
    
    # CSV 읽기 및 병합
    df_list = []
    for file in csv_files:
        try:
            df = pd.read_csv(file, encoding=encoding)
            df_list.append(df)
        except Exception as e:
            print(f"파일 {file} 읽기 실패: {e}")
    
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"✅ 총 {len(df_list)}개의 파일을 병합했습니다.")
    return combined_df

def convert_weekday(df):
    # 3. 요일 영문 -> 한국어 매핑
    weekday_map = {
        'Monday': '월',
        'Tuesday': '화',
        'Wednesday': '수',
        'Thursday': '목',
        'Friday': '금',
        'Saturday': '토',
        'Sunday': '일'
    }
    df['요일'] = df['대여일시2'].dt.day_name().map(weekday_map)

def add_weekend_and_time_period(df, datetime_col, weekend_col='is_weekend', time_col='time_period'):
    """
    주어진 datetime 칼럼에서 주말 여부와 시간대(오전/오후/저녁/심야)를 추출해 새로운 칼럼으로 추가합니다.

    Parameters:
        df (pd.DataFrame): 대상 데이터프레임
        datetime_col (str): datetime 형식의 칼럼명
        weekend_col (str): 생성할 주말 여부 칼럼명 (기본값: 'is_weekend')
        time_col (str): 생성할 시간대 칼럼명 (기본값: 'time_period')

    Returns:
        pd.DataFrame: 처리된 데이터프레임
    """
    # 주말 여부 (토/일)
    df[weekend_col] = df[datetime_col].dt.weekday >= 5  # True if Saturday(5) or Sunday(6)

    # 시간대 추출 (기준은 시(hour))
    def get_time_period(hour):
        if 5 <= hour < 12:
            return '오전'
        elif 12 <= hour < 17:
            return '오후'
        elif 17 <= hour < 21:
            return '저녁'
        else:
            return '심야'
    
    df[time_col] = df[datetime_col].dt.hour.apply(get_time_period)
    
    return df

In [ ]:
# 함수 호출 예시
bikes = merge_csv_files('./data')

# 결과 확인
print(bikes.head())

bikes['대여일시2'] = pd.to_datetime(bikes['대여일시'])
convert_weekday(bikes)
bikes = add_weekend_and_time_period(bikes, '대여일시2')
bikes['이용시간2'] = bikes['이용시간'].apply(lambda x: x/60)


In [ ]:
bike_shop = pd.read_excel('./data/bike_rent_location.xlsx')

bike_shop.head(1)

In [ ]:
bikes = pd.merge(bikes, bike_shop, left_on='대여 대여소번호', right_on='대여소번호')

In [ ]:
bikes.drop(['대여소번호','대여소명'], axis='columns', inplace=True)

In [ ]:
bikes.rename(columns={'구분' : '대여구','위도' : '대여점위도','경도' : '대여점경도'}, inplace=True)
bikes.head()

In [ ]:
sns.countplot(x='요일', data=bikes, order=['월','화','수','목','금','토','일'])

In [ ]:
plt.figure(figsize=(15, 4))


In [ ]:
bikes['일자'] = bikes['대여일시2'].dt.day
sns.countplot(x='일자', data=bikes)


In [ ]:
bikes['대여시간대'] = bikes['대여일시2'].dt.hour
sns.countplot(x='대여시간대', data=bikes)

In [ ]:
hourly_dayofweek_ride = bikes.pivot_table(index='대여시간대', columns='요일', values='자전거번호', aggfunc='count')

hourly_dayofweek_ride.plot(kind='line')